In [1]:
import os

os.environ['OPENAI_API_KEY'] = ''

In [2]:
def total_assets_tool(input_text: str) -> str:
    try:
        current_assets, non_current_assets = map(float, input_text.split(","))
        total_assets = current_assets - non_current_assets
        
        return f"Total assets are: {total_assets}"
    except Exception as e:
        return f"Error: {str(e)}"

def total_liabilities_tool(input_text: str) -> str:
    try:
        current_liabilities, non_current_liabilities = map(float, input_text.split(","))
        total_liabilities = current_liabilities + non_current_liabilities
        
        return f"Total liabilities are: {total_liabilities}"
    except Exception as e:
        return f"Error: {str(e)}"

In [3]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.5)

In [24]:
from langchain.prompts import PromptTemplate

# Create a prompt template that includes the necessary variables, including intermediate_steps
prompt_template = """
You are an intelligent assistant that can perform financial calculations based on user input.
You have access to the following tools:

{tool_names}

{tools}

Based on the user's input, choose the correct tool to provide the necessary calculations.

{intermediate_steps}
{agent_scratchpad}
User's input: {input_text}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["input_text", "tools", "tool_names", "intermediate_steps", "agent_scratchpad"]
)


In [12]:
from langchain.prompts import PromptTemplate

# Create a prompt template that includes the necessary variables
prompt_template = """
You are an intelligent assistant that can perform financial calculations based on user input.
You have access to the following tools:

{tool_names}

{tools}

Based on the user's input, choose the correct tool to provide the necessary calculations.

{agent_scratchpad}
User's input: {input_text}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["input_text", "tools", "tool_names", "agent_scratchpad"]
)

In [25]:
from langchain.agents import create_react_agent, Tool

total_assets_function_tool = Tool(
    name="Calculate Total Assets",
    func=total_assets_tool,
    description="Calculates total assets given current assets and non-current assets."
)
total_liabilities_function_tool = Tool(
    name="Calculate Total Liabilities",
    func=total_liabilities_tool,
    description="Calculates total liabilities given current liabilities and non-current liabilities."
)

agent = create_react_agent(
    tools=[total_assets_function_tool, total_liabilities_function_tool],
    llm=llm,
    prompt=prompt
)

In [27]:
user_input_assets = "Calculate total assets with 1000 current and 2000 non-current assets."
user_input_liabilities = "Calculate total liabilities with 500 current and 1500 non-current liabilities."

response_assets = agent.invoke({"input_text": user_input_assets, "intermediate_steps": []})
print(response_assets)

OutputParserException: Could not parse LLM output: `
Output: Total assets = $3000`

In [ ]:
response_liabilities = agent.{"input_text": user_input_assets}
print(response_liabilities)

In [28]:
from langchain.prompts import PromptTemplate

# Adjust the prompt template to specify the expected output format
prompt_template = """
You are an intelligent assistant that can perform financial calculations based on user input.
You have access to the following tools:

{tool_names}

{tools}

Based on the user's input, choose the correct tool to provide the necessary calculations.

{intermediate_steps}
{agent_scratchpad}

User's input: {input_text}

Please output the result in the following format: "Result: [calculation result]".
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["input_text", "tools", "tool_names", "intermediate_steps", "agent_scratchpad"]
)

In [32]:
from langchain.schema import BaseOutputParser
from langchain.agents import create_react_agent, Tool
from langchain_openai import OpenAI
from langchain.schema import OutputParserException

# Custom output parser class
class CustomOutputParser(BaseOutputParser):
    def parse(self, text: str):
        try:
            # Custom parsing logic - this is a simple example
            if "Total assets" in text:
                return {"result": text.split("=")[-1].strip()}
            elif "Total liabilities" in text:
                return {"result": text.split("=")[-1].strip()}
            else:
                raise ValueError("Could not parse the output")
        except Exception as e:
            raise OutputParserException(f"Could not parse LLM output: `{text}`")

# Create the tools
total_assets_function_tool = Tool(
    name="Calculate Total Assets",
    func=total_assets_tool,
    description="Calculates total assets given current assets and non-current assets."
)

total_liabilities_function_tool = Tool(
    name="Calculate Total Liabilities",
    func=total_liabilities_tool,
    description="Calculates total liabilities given current liabilities and non-current liabilities."
)

# Initialize the LLM
llm = OpenAI(temperature=0)

# Use the custom output parser
output_parser = CustomOutputParser()

# Create the agent
agent = create_react_agent(
    tools=[total_assets_function_tool, total_liabilities_function_tool],
    llm=llm,
    prompt=prompt,
    output_parser=output_parser
)

# Run the agent using invoke
response_assets = agent.invoke({"input_text": user_input_assets, "intermediate_steps": []})
print(response_assets)

response_liabilities = agent.invoke({"input_text": user_input_liabilities, "intermediate_steps": []})
print(response_liabilities)

OutputParserException: Could not parse LLM output: `
Result: 3000`